In [1]:
import requests
from pprint import pprint
import re
import lxml.html
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def log_error(line1, line2):
    _line1 = str(line1)
    _line2 = str(line2)
    diff = 0
    chars1 = len(_line1) + 10
    chars2 = len(_line2) + 10
    if chars1 > chars2:
        diff = chars1 - chars2
        if diff % 2 != 0:
            diff = int(diff - .5)
            _line2 = '# ' + _line2
        else:
            _line2 = ' ' + _line2
        main_hashes = ''.join(['#' for i in range(chars1)])
        hashes = ''.join(['#' for i in range(int(diff/2)+4)])
        log_line = (f"{main_hashes}\n"
                    f"#### {_line1} ####\n"
                    f"{hashes}{_line2} {hashes}\n"                    
                    f"{main_hashes}\n")
        return(log_line)
   
    elif chars2 > chars1:
        diff = chars2 - chars1
        if diff % 2 != 0:
            diff = int(diff - .5)
            _line1 = '# ' + _line1
        else:
            _line1 = ' ' + _line1
        main_hashes = ''.join(['#' for i in range(chars2)])
        hashes = ''.join(['#' for i in range(int(diff/2)+4)])
        log_line = (f"{main_hashes}\n"
                    
                    f"{hashes}{_line1} {hashes}\n"
                    f"#### {_line2} ####\n"
                    f"{main_hashes}\n")
        return(log_line)
        
    else:
        hashes = '####'
        main_hashes = ''.join(['#' for i in range(chars2)])
        log_line = (f"{main_hashes}\n"
                    f"#### {_line1} ####\n"
                    f"{hashes} {_line2} {hashes}\n"
                    f"{main_hashes}\n")
        return(log_line)
        

In [3]:
# init session 
S = requests.Session()

# set URL
URL = "https://en.wikipedia.org/w/api.php"

In [4]:
page_id = "10138926"
topic = 'city_region'
subtopic = 'italy'

In [5]:
# get sections
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "sections"
}

sectionsR = S.get(url=URL, params=PARAMS)
sectionDATA = sectionsR.json()



In [6]:
sectionDATA

{'parse': {'title': 'List of cities in Italy',
  'pageid': 10138926,
  'sections': [{'toclevel': 1,
    'level': '2',
    'line': 'Cities',
    'number': '1',
    'index': '1',
    'fromtitle': 'List_of_cities_in_Italy',
    'byteoffset': 990,
    'anchor': 'Cities'},
   {'toclevel': 1,
    'level': '2',
    'line': 'Gallery',
    'number': '2',
    'index': '2',
    'fromtitle': 'List_of_cities_in_Italy',
    'byteoffset': 17851,
    'anchor': 'Gallery'},
   {'toclevel': 1,
    'level': '2',
    'line': 'Map of the cities',
    'number': '3',
    'index': '3',
    'fromtitle': 'List_of_cities_in_Italy',
    'byteoffset': 19289,
    'anchor': 'Map_of_the_cities'},
   {'toclevel': 1,
    'level': '2',
    'line': 'See also',
    'number': '4',
    'index': '4',
    'fromtitle': 'List_of_cities_in_Italy',
    'byteoffset': 21276,
    'anchor': 'See_also'},
   {'toclevel': 1,
    'level': '2',
    'line': 'References',
    'number': '5',
    'index': '5',
    'fromtitle': 'List_of_cities_

In [ ]:
sections = [sec['anchor'] for sec in sectionDATA["parse"]["sections"]]

In [7]:
# create section list filtering out non-data sections
filt = [
    'References',
    'See_also',
    'External_links',
    #"Historical_names_of_Valletta's_streets_and_squares"
]

incl = [
    'Cities'
]

# sec_list = [sec for sec in sectionDATA["parse"]["sections"] if not sec['anchor'] in filt]
sec_list = [sec for sec in sectionDATA["parse"]["sections"] if sec['anchor'] in incl]


len(sec_list)
sec_list

[{'toclevel': 1,
  'level': '2',
  'line': 'Cities',
  'number': '1',
  'index': '1',
  'fromtitle': 'List_of_cities_in_Italy',
  'byteoffset': 990,
  'anchor': 'Cities'}]

In [ ]:
for index, line in enumerate(lines):
    print(index, line)

In [100]:
# create nested section_dict dictionary from section list based on toclevel
# refactored
section_dict = {}
current_parent = None
for i in range(len(sec_list)):
    if i == 0 and len(sec_list) > 1:
        if sec_list[i]['toclevel'] < sec_list[i + 1]['toclevel']:
            section_dict[sec_list[i]['line']] = {}
            current_grandparent = sec_list[i]['line']
        else:
            section_dict[sec_list[i]['line']] = []

    elif i != len(sec_list) - 1:
        if sec_list[i - 1]['toclevel'] < sec_list[i]['toclevel'] and sec_list[i]['toclevel'] < sec_list[i + 1]['toclevel']:
            section_dict[current_grandparent][sec_list[i]['line']] = {}
            current_parent = section_dict[current_grandparent][sec_list[i]['line']]
            
        elif sec_list[i]['toclevel'] == sec_list[i + 1]['toclevel'] or sec_list[i]['toclevel'] > sec_list[i + 1]['toclevel']:
            if current_parent != None:
                current_parent[sec_list[i]['line']] = []
            else: section_dict[sec_list[i]['line']] = []
            
        elif sec_list[i]['toclevel'] < sec_list[i + 1]['toclevel']:
            section_dict[current_grandparent][sec_list[i]['line']] = {}
            current_parent = section_dict[current_grandparent][sec_list[i]['line']]
            
    else:
        if sec_list[i]['toclevel'] == sec_list[i - 1]['toclevel']:
            section_dict[sec_list[i]['line']] = []
        else:
            section_dict[sec_list[i]['line']] = []
              

#pprint(section_dict)

In [101]:
section_dict

{'Cities': []}

In [52]:
# hard coded name for section in section dict bec inconsistent wikipedia format
section_dict = {'San Jose': []}

In [ ]:
# create nested section_dict dictionary from section list based on toclevel
# adding 'Z' section by hand consider refactor
section_dict = {}
is_new_parent = True
parent = ''
for i in range(len(sec_list) - 1):
    if sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 1:
        section_dict[sec_list[i]['anchor']] = []
    elif sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 2:
        section_dict[sec_list[i]['anchor']] = {}
        parent = sec_list[i]['anchor']
    elif sec_list[i]['toclevel'] == 2:
        section_dict[parent][sec_list[i]['anchor']] = []
section_dict['Z'] = []
#section_dict

In [ ]:
# make list of top level sections
secs = [sec for sec, val in section_dict.items()]


In [ ]:
# make section dict if no nested sections
section_dict = {sec['anchor']:[] for sec in sec_list}

In [ ]:
# get links
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "links"
}

linkR = S.get(url=URL, params=PARAMS)
linkDATA = linkR.json()

pprint(linkDATA["parse"]["links"])

In [ ]:
links = [link['*'] for link in linkDATA["parse"]["links"]]
for i, link in enumerate(links):
    print(i, link)

In [ ]:
# visualize link data
for link in links[3:4]:
    # get page info
    PARAMS = {
        "action": "query",
        "titles": link,
        "format": "json",
        "prop": "info",
        'inprop': "displaytitle"
    }

    infoR = S.get(url=URL, params=PARAMS)
    infoDATA = infoR.json()
    pprint(infoDATA)

In [102]:
# get data
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "wikitext"
}

textR = S.get(url=URL, params=PARAMS)
textDATA = textR.json()


In [103]:
# parse data

lines = textDATA["parse"]["wikitext"]["*"].split('\n')

In [104]:
for index, line in enumerate(lines[:301]):
    print(index, line)

0 The following is a '''list of Italian ''[[comuni]]''''' (municipalities) with a population over 50,000. The table below contains the cities populations as of December 31, 2018,<ref>{{cite web|url=http://demo.istat.it/bilmens2018gen/index.html|title=Demographic Balance for the year 20178 and Resident Population on 31st December|last=|first=|date=|website=demo.istat.it|url-status=live|archive-url=|archive-date=|accessdate=13 June 2018}}</ref> as estimated by the Italian [[Italian National Institute of Statistics]]<ref>{{cite web|url=https://www.istat.it/en/|title=Istat.it english|website=www.istat.it|accessdate=29 April 2018|url-status=live|archiveurl=https://web.archive.org/web/20180322020123/https://www.istat.it/en/|archivedate=22 March 2018}}</ref>, and the cities [[census]] population from the [[2011 Italian Census]] <ref>{{Cite web|url=http://demo.istat.it/bilmens20111009/index.html|title=Statistiche demografiche ISTAT|website=demo.istat.it|access-date=2019-11-25}}</ref>
1 
2 == C

In [84]:
this_sec_name

'Cities'

In [111]:
# parse 
#       WikiText
# data


debug = False
special = True

this_sec_name = None
this_sec = None

this_item = {}

new_cols = False
cols = []

for index, line in enumerate(lines[:301]):
    #print(line)
    if debug:
        if index != 0: 
            print('$\n' + str(index -1) + '\n' + lines[index - 1] + '\n' + str(index) + '\n' + line + '\n$')
            print(index, re.match('\|\-', lines[index - 1][0:2]))
            print(line)
            print(index,re.match('\!\w+', line[0:2]))

    if re.match('==\s\w', line[0:4]):
        if debug or special: print("$$-2-$$")
        this_sec_name = line.strip().replace("=", "").strip()
        print(this_sec_name)

    if re.match('\{\|', lines[index - 1][0:2]) and re.match('\!\w+', line[0:2]):
        if debug: print("$-6-$$")
        cols = []
        new_cols = True

    if re.match('\!\w+', lines[index - 1][0:2]) and re.match('\|\-', line[0:2]):
        if debug: print("$-7-$$")
        new_cols = False
        this_sec = []
        #this_sec_start = index + 2

    if new_cols == True:
        if debug: print("$-8-$$")
        cols.append(re.search(r'\!(\w+)', line).group(1))
        #print(cols)

    if this_sec != None:
        if debug or special: print("$-10-$$")
        if re.match('\|a', lines[index][0:2]):
            print(index)
            if debug or special: print("$-11-$$")
            this_line = re.search('\|align=center\|(.*)', line).group(1)
            print(this_line)
            this_item = {col: '' for col in cols}
            this_line = this_line.split("||")
            this_item[cols[0]] = this_line[0]
            if '|' in this_line[1]:
                
                print("pipe")
                print(this_line[1])
                print(index)
            this_item[cols[1]] = re.search(r'\[\[.*\|(.*)\]\]', this_line[1]).group(1) if '|' in re.search(r'\[\[(.*)\]\]', this_line[1]).group(1) else re.search(r'\[\[(.*)\]\]', this_line[1]).group(1)   
            this_item[cols[2]] = int(re.sub(r'\{|\}', '', this_line[2]).split('|')[1])
            this_item[cols[3]] = int(re.sub(r'\{|\}', '', this_line[2]).split('|')[2])
            this_item[cols[4]] = f"{round(((this_item[cols[3]] - this_item[cols[2]]) / this_item[cols[2]] * 10))}%"
            this_item[cols[5]] = re.search(r'\{\{.*\|(.*)\}\}', this_line[3]).group(1)
            this_sec.append(this_item)
            section_dict[this_sec_name] = this_sec
            #print(this_item)

    # if this_item != None:

              
                   
                    
                    
    

$$-2-$$
Cities
$-10-$$
$-10-$$
11
$-11-$$
 1 || align="left" | [[Rome]] || {{change|2617175|2856133}} || align="left" | {{flag|Lazio}}
pipe
 align="left" | [[Rome]] 
11
$-10-$$
$-10-$$
13
$-11-$$
 2 || align="left" | [[Milan]] || {{change|1242123|1378689}} || align="left" | {{flag|Lombardy}}
pipe
 align="left" | [[Milan]] 
13
$-10-$$
$-10-$$
15
$-11-$$
 3 || align="left" | [[Naples]] || {{change|962003|959188}} || align="left" | {{flag|Campania}}
pipe
 align="left" | [[Naples]] 
15
$-10-$$
$-10-$$
17
$-11-$$
 4 || align="left" | [[Turin]] || {{change|872367|875698}} || align="left" | {{flag|Piedmont}}
pipe
 align="left" | [[Turin]] 
17
$-10-$$
$-10-$$
19
$-11-$$
 5 || align="left" | [[Palermo]] || {{change|657651|663401}} || align="left" | {{flag|Sicily}}
pipe
 align="left" | [[Palermo]] 
19
$-10-$$
$-10-$$
21
$-11-$$
 6 || align="left" | [[Genoa]] || {{change|586180|578000}} || align="left" | {{flag|Liguria}} 
pipe
 align="left" | [[Genoa]] 
21
$-10-$$
$-10-$$
23
$-11-$$
 7 || align="

In [112]:
section_dict

{'Cities': [{'Rank': ' 1 ',
   'City': 'Rome',
   '2011': 2617175,
   '2018': 2856133,
   'Change': '1%',
   'Region': 'Lazio'},
  {'Rank': ' 2 ',
   'City': 'Milan',
   '2011': 1242123,
   '2018': 1378689,
   'Change': '1%',
   'Region': 'Lombardy'},
  {'Rank': ' 3 ',
   'City': 'Naples',
   '2011': 962003,
   '2018': 959188,
   'Change': '0%',
   'Region': 'Campania'},
  {'Rank': ' 4 ',
   'City': 'Turin',
   '2011': 872367,
   '2018': 875698,
   'Change': '0%',
   'Region': 'Piedmont'},
  {'Rank': ' 5 ',
   'City': 'Palermo',
   '2011': 657651,
   '2018': 663401,
   'Change': '0%',
   'Region': 'Sicily'},
  {'Rank': ' 6 ',
   'City': 'Genoa',
   '2011': 586180,
   '2018': 578000,
   'Change': '0%',
   'Region': 'Liguria'},
  {'Rank': ' 7 ',
   'City': 'Bologna',
   '2011': 371337,
   '2018': 390636,
   'Change': '1%',
   'Region': 'Emilia-Romagna'},
  {'Rank': ' 8 ',
   'City': 'Florence',
   '2011': 358079,
   '2018': 378839,
   'Change': '1%',
   'Region': 'Tuscany'},
  {'Rank': '

In [124]:
# items in cities table
total = sum([city['2018']for city in section_dict['Cities']])
cities = [city['City'] + ', ' + city['Region']  + ', ' + str(round(city['2018']/total, 3)) for city in section_dict['Cities']]
cities

['Rome, Lazio, 0.137',
 'Milan, Lombardy, 0.066',
 'Naples, Campania, 0.046',
 'Turin, Piedmont, 0.042',
 'Palermo, Sicily, 0.032',
 'Genoa, Liguria, 0.028',
 'Bologna, Emilia-Romagna, 0.019',
 'Florence, Tuscany, 0.018',
 'Bari, Apulia, 0.015',
 'Catania, Sicily, 0.015',
 'Venice, Veneto, 0.012',
 'Verona, Veneto, 0.012',
 'Messina, Sicily, 0.011',
 'Padua, Veneto, 0.01',
 'Trieste, Friuli-Venezia Giulia, 0.01',
 'Brescia, Lombardy, 0.01',
 'Taranto, Apulia, 0.009',
 'Parma, Emilia-Romagna, 0.009',
 'Prato, Tuscany, 0.009',
 'Modena, Emilia-Romagna, 0.009',
 'Reggio Calabria, Calabria, 0.009',
 'Reggio Emilia, Emilia-Romagna, 0.008',
 'Perugia, Umbria, 0.008',
 'Livorno, Tuscany, 0.008',
 'Ravenna, Emilia-Romagna, 0.008',
 'Cagliari, Sardinia, 0.007',
 'Foggia, Apulia, 0.007',
 'Rimini, Emilia-Romagna, 0.007',
 'Salerno, Campania, 0.006',
 'Ferrara, Emilia-Romagna, 0.006',
 'Sassari, Sardinia, 0.006',
 'Latina, Lazio, 0.006',
 'Giugliano in Campania, Campania, 0.006',
 'Monza, Lombard

In [33]:
# make 1 list
all_items = [item for section, items in section_dict.items() for item in items]
# make 2 lists, nested
# all_items = [item for section, item in section_dict.items()]
# flatten lists 2 ways
# all_items = [lyst1 for lyst2 in all_items for lyst1 in lyst2]
#all_items = [item for items in all_items for item in items]
all_items 

['19th Avenue (San Francisco)|19th Avenue',
 'California Street (San Francisco)|California Street',
 'Fell Street',
 'Geary Boulevard',
 'Grant Avenue',
 'Fulton Street(San Francisco)|Fulton Street',
 'Lincoln Way (San Francisco)|Lincoln Way',
 'Lombard Street (San Francisco)|Lombard Street',
 'Market Street (San Francisco)|Market Street',
 'Park Presidio Boulevard',
 'Portola Drive',
 'Van Ness Avenue (San Francisco)|Van Ness Avenue',
 '24th Street (San Francisco)|24th Street',
 'Columbus Avenue (San Francisco)|Columbus Avenue',
 'Fillmore Street',
 'Kearny Street',
 'Mission Street',
 'Polk Street',
 'Stockton Street (San Francisco)|Stockton Street',
 'Union Street (San Francisco)|Union Street',
 'Third Street (San Francisco)|Third Street',
 '22nd Street (San Francisco)',
 '49-Mile Scenic Drive',
 'Alemany Boulevard',
 'Broadway (San Francisco)|Broadway',
 'The Castro, San Francisco|Castro Street',
 'Cesar Chavez Street',
 'Divisadero Street',
 'Don Chee Way',
 'Embarcadero (San Fran

In [25]:
with open(f"../output/street_type_designations.txt", 'r+', encoding="utf-8") as filepath:
    doc = filepath.read()
street_types = doc.split('\n')
street_types

['Highway',
 'Autobahn',
 'Auto-estrada',
 'Autoroute',
 'Autostrada',
 'Autostrasse',
 'Bypass',
 'Expressway',
 'Freeway',
 'Motorway',
 'Pike',
 'Turnpike',
 'Avenue',
 'Boulevard',
 'Parade',
 'Road',
 'Street',
 'Arcade',
 'Alley',
 'Bay',
 'Branch',
 'Brook',
 'Burg',
 'Byway',
 'Camp',
 'Center',
 'Club',
 'Common',
 'Corner',
 'Course',
 'Dale',
 'Divide',
 'Drive',
 'Estate',
 'Flat',
 'Forge',
 'Fork',
 'Fort',
 'Gardens',
 'Gate',
 'Gateway',
 'Glen',
 'Green',
 'Grove',
 'Harbor',
 'Haven',
 'Heights',
 'Highlands',
 'Hollow',
 'Key',
 'Knoll',
 'Landing',
 'Lane',
 'Light',
 'Loaf',
 'Lock',
 'Lodge',
 'Manor',
 'Meadow',
 'Mews',
 'Mill',
 'Mission',
 'Neck',
 'Orchard',
 'Passage',
 'Path',
 'Pathway',
 'Ranch',
 'Rapid',
 'Rest',
 'Route',
 'Row',
 'Rue',
 'Run',
 'Station',
 'Terrace',
 'Throughway',
 'Trace',
 'Track',
 'Trafficway',
 'Trail',
 'Trailer',
 'Union',
 'Vale',
 'View',
 'Village',
 'Ville',
 'Vista',
 'Walk',
 'Wall',
 'Way',
 'Well',
 'Wynd',
 'Cul-de-s

In [43]:
all_items = []
for section, items in section_dict.items():
    for index, item in enumerate(items):
        #print(item)
        item = re.sub(r'\[|\]', '', item)
        if '<ref' in item:
            #print(item)
            all_items.append(re.search(r'(.*)\<ref', item).group(1))
        elif '|' in item:
            #print(item)
            #all_items.append(re.search(r'\|(.*)', item).group(1))
            for ytem in item.split('|'):
                all_items.append(ytem.strip())
        elif 'and ' in item:
            print(f"### and-split {index}")
            if ',' in item:
                #print(f"comma-split{item.split(',')}")
                item = re.sub(r'and ', ',', item)
                #print(f"### comma-split {index}")
                for ytem in item.split(','):
                    all_items.append(ytem.strip())
            else:
                #print(f"and-split{item.split('and')}")
                #print(f"### and-split {index}")
                for ytem in item.split('and '):
                    all_items.append(ytem.strip())
        elif '& ' in item:
            for ytem in item.split('& '):
                    all_items.append(ytem.strip())
        elif ',' in item:
            #print(f"comma-split{item.split(',')}")
            print(f"### comma-split {index}")
            for ytem in item.split(','):
                all_items.append(ytem.strip())

        else:
            all_items.append(item)


In [44]:
#for index,item in enumerate(all_items): print(index, item)

In [45]:
for index, item in enumerate(all_items):
    if '#' in item:
        #print(item)
        #print([ytem for ytem in item.split('#')])
        #print(all_items[:index])
        #print(all_items[index + 1:])
        all_items = all_items[:index-1] + [ytem for ytem in item.split('#')] + all_items[index + 1:]
    if ',' in item:
        all_items = all_items[:index-1] + [ytem for ytem in item.split(',')] + all_items[index + 1:]
            
all_items

['19th Avenue (San Francisco)',
 '19th Avenue',
 'California Street (San Francisco)',
 'California Street',
 'Fell Street',
 'Geary Boulevard',
 'Grant Avenue',
 'Fulton Street(San Francisco)',
 'Fulton Street',
 'Lincoln Way (San Francisco)',
 'Lincoln Way',
 'Lombard Street (San Francisco)',
 'Lombard Street',
 'Market Street (San Francisco)',
 'Market Street',
 'Park Presidio Boulevard',
 'Portola Drive',
 'Van Ness Avenue (San Francisco)',
 'Van Ness Avenue',
 '24th Street (San Francisco)',
 '24th Street',
 'Columbus Avenue (San Francisco)',
 'Columbus Avenue',
 'Fillmore Street',
 'Kearny Street',
 'Mission Street',
 'Polk Street',
 'Stockton Street (San Francisco)',
 'Stockton Street',
 'Union Street (San Francisco)',
 'Union Street',
 'Third Street (San Francisco)',
 'Third Street',
 '22nd Street (San Francisco)',
 '49-Mile Scenic Drive',
 'Alemany Boulevard',
 'Broadway (San Francisco)',
 'The Castro',
 ' San Francisco',
 'Castro Street',
 'Cesar Chavez Street',
 'Divisadero St

In [46]:
if '#' in 'Jean Parisot de Valette#Jean De Valette Square': print('yes')

yes


In [47]:
for index, item in enumerate(all_items):
    #remove digits
    item = re.sub('\d+$', '', item).strip()
    # remove comments in parens
    item = re.sub(r'\((.*)\)', '', item).strip()
    # check if last word is street type
    # sometimes both words in street types will have to revalidate when creating
    if len(item.split(' ')) > 1:
        last_word = item.split(' ')[-1]
    if last_word in street_types:
        #print(last_word)
        #print(item)
        all_items[index] = item.replace(last_word, '').strip()
    else:
        all_items[index] = item
        
        #print(item)
all_items

['19th',
 '19th',
 'California',
 'California',
 'Fell',
 'Geary',
 'Grant',
 'Fulton',
 'Fulton',
 'Lincoln',
 'Lincoln',
 'Lombard',
 'Lombard',
 'Market',
 'Market',
 'Park Presidio',
 'Portola',
 'Van Ness',
 'Van Ness',
 '24th',
 '24th',
 'Columbus',
 'Columbus',
 'Fillmore',
 'Kearny',
 'Mission',
 'Polk',
 'Stockton',
 'Stockton',
 'Union',
 'Union',
 'Third',
 'Third',
 '22nd',
 '49-Mile Scenic',
 'Alemany',
 'Broadway',
 'The Castro',
 'San Francisco',
 'Castro',
 'Cesar Chavez',
 'Divisadero',
 'Don Chee',
 'Embarcadero',
 'The Embarcadero',
 'Filbert',
 'Filbert',
 'Golden Gate',
 'Great',
 'Haight',
 'Hayes',
 'Junipero Serra',
 'John F. Kennedy',
 'Lombard',
 'Lombard',
 'Montgomery',
 'New Montgomery',
 'Octavia',
 'California State',
 'Skyline',
 'Vermont',
 'Vermont',
 'Howard',
 'Balmy',
 'Belden',
 'Clarion',
 'Jack Kerouac',
 'Macondray',
 'Maiden',
 'Maiden']

In [48]:
dedupe_all_items = []
for item in all_items:
    if item not in dedupe_all_items:
        dedupe_all_items.append(item)
dedupe_all_items

['19th',
 'California',
 'Fell',
 'Geary',
 'Grant',
 'Fulton',
 'Lincoln',
 'Lombard',
 'Market',
 'Park Presidio',
 'Portola',
 'Van Ness',
 '24th',
 'Columbus',
 'Fillmore',
 'Kearny',
 'Mission',
 'Polk',
 'Stockton',
 'Union',
 'Third',
 '22nd',
 '49-Mile Scenic',
 'Alemany',
 'Broadway',
 'The Castro',
 'San Francisco',
 'Castro',
 'Cesar Chavez',
 'Divisadero',
 'Don Chee',
 'Embarcadero',
 'The Embarcadero',
 'Filbert',
 'Golden Gate',
 'Great',
 'Haight',
 'Hayes',
 'Junipero Serra',
 'John F. Kennedy',
 'Montgomery',
 'New Montgomery',
 'Octavia',
 'California State',
 'Skyline',
 'Vermont',
 'Howard',
 'Balmy',
 'Belden',
 'Clarion',
 'Jack Kerouac',
 'Macondray',
 'Maiden']

In [49]:
# remove empty string if caused eg 'Pellier Avenue & Court'
dedupe_all_items = [item for item in dedupe_all_items if item != '']
dedupe_all_items

['19th',
 'California',
 'Fell',
 'Geary',
 'Grant',
 'Fulton',
 'Lincoln',
 'Lombard',
 'Market',
 'Park Presidio',
 'Portola',
 'Van Ness',
 '24th',
 'Columbus',
 'Fillmore',
 'Kearny',
 'Mission',
 'Polk',
 'Stockton',
 'Union',
 'Third',
 '22nd',
 '49-Mile Scenic',
 'Alemany',
 'Broadway',
 'The Castro',
 'San Francisco',
 'Castro',
 'Cesar Chavez',
 'Divisadero',
 'Don Chee',
 'Embarcadero',
 'The Embarcadero',
 'Filbert',
 'Golden Gate',
 'Great',
 'Haight',
 'Hayes',
 'Junipero Serra',
 'John F. Kennedy',
 'Montgomery',
 'New Montgomery',
 'Octavia',
 'California State',
 'Skyline',
 'Vermont',
 'Howard',
 'Balmy',
 'Belden',
 'Clarion',
 'Jack Kerouac',
 'Macondray',
 'Maiden']

In [114]:
limit = 'all'
with open(f"../output/{topic}_{subtopic}_{limit}.txt", 'w+', encoding="utf-8") as fp:
    for item in cities:
        fp.write(item + '\n')

In [ ]:
# use if items all are/begin with list of links [[link]]

items = []
descriptions = []
with open(topic + '_error_log.txt', 'w+', encoding="utf-8") as filepath:
    for index, line in enumerate(lines[77:291]):
        
        if re.match('\*\s*\[\[', line[0:5]):
            #print("%%%Match%%%")
            #print(line)
            try:
                line = re.sub(r'\<ref.*?\</ref>|\<ref.*\/>', '', line)
                
            except:
                topic_error_line = f"{topic} line sub parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            try:
                
                item = re.search(r'\[\[(.*?)\]\]', line).group(1)
                
            except:
                topic_error_line = f"{topic} item parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            try:
                
                description = re.search(r'\]\]\:*(.*)', line).group(1)
                
                
            except:
                topic_error_line = f"{topic} description parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            items.append(item)
            descriptions.append(description)
        else:
            continue
            #print("%%%No%%%")
            #print(line)

In [ ]:
items = [re.sub(r'\:wiktionary\:', '', item) for item in items]
items = [re.sub(r'\|.*', '', item) for item in items]
items

In [ ]:
# use to find info page for each item/link and rank 
item_infos = []
for index, title in enumerate(items):
    with open(topic + '_error_log.txt', 'w+', encoding="utf-8") as filepath:
        
        url = 'https://en.wikipedia.org/w/index.php?title=' + title + '&action=info'

        #print(url)
        response = requests.get(url)
        doc = lxml.html.fromstring(response.content)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        try:
            # info_table = doc.xpath('//*[@class="wikitable mw-page-info"]')[0]
            watchers = doc.xpath('//*[@id="mw-pageinfo-watchers"]')[0][1].text
            
        except:
            topic_error_line = f"watchers parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            watchers = "#error#"

        try:
            redirects = doc.xpath('//*[contains(text(),"redirects")]/../../td')[1].text
            #redirects = doc.xpath('//*[contains(text(),"redirects")]/parent::node()/parent::node()')
            #redirects = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[2]/tr[9]/td')[1].text
            
        except:
            topic_error_line = f"redirects parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            redirects = "#error#"
             
        try:
            views = doc.xpath('//*[@id="mw-pvi-month-count"]/td/div')[0].text
            
        except:
            topic_error_line = f"views parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            views = "#error#"        
        
        try:
            edits = doc.xpath('//*[@id="mw-pageinfo-edits"]/td')[1].text
            #edits = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[4]/tr[5]/td')[1].text
    
        except:
            topic_error_line = f"edits parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            edits = "#error#"
        
        item_infos.append({
            topic: title,
            'watchers': watchers,
            'redirects': redirects,
            'views': views,
            'edits': edits
        })


In [ ]:
# transfer item_infos to json file
import json
with open(topic + '_infos.json', 'w') as filepath:
    json.dump(item_infos, filepath)

In [ ]:
error_items = [item[topic] for item in item_infos if item['views'] == '#error#']

for index, title in enumerate(error_items):
    with open('info_error_log.txt', 'w+', encoding="utf-8") as filepath:
        #print(title)
        del_idx = 0
        for index, item in enumerate(item_infos):
            if item[topic] == title:
                del_idx = index
                #del item_infos[index]
        #print(item)
        
        
        if re.match(r'\W*', title):
            print('%%%')
            search = re.search(r'(\w+)?', title).group(1)
            print(search)
            title = search
        
        print(title)
        
        url = 'https://en.wikipedia.org/w/index.php?title=' + title + '&action=info'

        #print(url)
        response = requests.get(url)
        doc = lxml.html.fromstring(response.content)
        soup = BeautifulSoup(response.content, 'html.parser')
  
        try:
            # info_table = doc.xpath('//*[@class="wikitable mw-page-info"]')[0]
            watchers = doc.xpath('//*[@id="mw-pageinfo-watchers"]')[0][1].text
            
        except:
            topic_error_line = f"watchers parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            watchers = "#error#"

        try:
            redirects = doc.xpath('//*[contains(text(),"redirects")]/../../td')[1].text
            #redirects = doc.xpath('//*[contains(text(),"redirects")]/parent::node()/parent::node()')
            #redirects = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[2]/tr[9]/td')[1].text
            
        except:
            topic_error_line = f"redirects parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            redirects = "#error#"
             
        try:
            views = doc.xpath('//*[@id="mw-pvi-month-count"]/td/div')[0].text
            
        except:
            topic_error_line = f"views parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            views = "#error#"        
        
        try:
            edits = doc.xpath('//*[@id="mw-pageinfo-edits"]/td')[1].text
            #edits = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[4]/tr[5]/td')[1].text
    
        except:
            topic_error_line = f"edits parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            edits = "#error#"
            
        item_infos[del_idx] = {
            topic: title,
            'watchers': watchers,
            'redirects': redirects,
            'views': views,
            'edits': edits
        }

In [ ]:
with open('mineral_infos.json', 'r') as jfile:
    data=jfile.read()

# parse file
item_infos = json.loads(data)

In [ ]:
_item_infos = item_infos
len(_item_infos)

In [ ]:
#item_infos = [{item['views']: int(item['views'].replace(',', ''))} for item in item_infos]
for index, item in enumerate(_item_infos):
    if item['views'] != '#error#':
        if item['watchers'] == 'Fewer than 30 watchers':
            item['watchers'] = '29'
        item['views'] = int(item['views'].replace(',', ''))
        item['watchers'] = int(item['watchers'].replace(',', ''))
        #item['edits'] = int(item['edits'].replace(',', ''))
        
        
        

In [ ]:
by_views = sorted(_item_infos, key=lambda k : k['views'], reverse=True)
for index, item in enumerate(by_views):
    item['view_rank'] = index

In [ ]:
by_edits = sorted(_item_infos, key=lambda k : k['edits'], reverse=True)
for index, item in enumerate(by_edits):
    item['edit_rank'] = index

In [ ]:
by_watchers = sorted(_item_infos, key=lambda k : k['watchers'], reverse=True)
for index, item in enumerate(by_watchers):
    item['watcher_rank'] = index

In [ ]:
for item in _item_infos:
    #item['total_rank'] = (item['view_rank'] + item['edit_rank'] + item['watcher_rank']) / 3
    item['total_rank'] = (item['view_rank'] + item['watcher_rank']) / 2

    

In [ ]:
by_total = sorted(_item_infos, key=lambda k : k['total_rank'], reverse=True)

In [ ]:
dedupe = []
for item in by_total:
    if item[topic] not in dedupe:
        dedupe.append(item[topic])
        

In [ ]:
start = 0
limit = 44
top_items = dedupe[start:limit]

In [ ]:
with open(f"{topic}_{limit}.txt", 'w+', encoding="utf-8") as fp:
    for item in top_items:
        fp.write(item + '\n')

In [ ]:
min_df = pd.DataFrame(_item_infos)
min_df.head()
min_df.sort_values(by=['total_rank'], ascending=False).head(100)

In [ ]:
min_df = pd.DataFrame(_item_infos)
min_df.head()
min_df.sort_values(by=['views'], ascending=False).head(100)

In [ ]:
min_df.sort_values(by=['watchers'], ascending=False).head(100)

In [ ]:
min_df.sort_values(by=['edits'], ascending=False).head(100)

In [ ]:
# parse data

lines = textDATA["parse"]["wikitext"]["*"].split('\n')[3:-10] # [::-1]
this_sec = {}
sec = ''
sub_sec = ''
with open('error_lines.txt', 'w+', encoding="utf-8") as filepath:
    for index, line in enumerate(lines):
        if index == 23:
            print(line)
        if re.match('==\w', line[0:3]):
            sec = line.strip().replace("=", "")
            if len(this_sec) > 0:
                section_dict[secs[secs.index(sec) - 1]] = this_sec
            this_sec = {}
        if type(section_dict[sec]).__name__ == 'list':
            if re.match('\*\[\[', line[0:3]):
                try:
                    poet = re.search(r'\[\[(.*?)\]\]', line).group(1)
                    #print(poet)

                except:
                    print("##########################")
                    print("#### poet parse error ####")
                    print(poet_method, index, line)
                    filepath.write(line + '\n')
                    print(line[-2:])
                    print("##########################")

                date_m = re.search('\((.*)\)', line)

                try:
                    date = date_m.group(0)
                except:
                    print("##########################")
                    print("#### date parse error ####")
                    print(index, line)
                    filepath.write(line + '\n')
                    date = ""
                    print("##########################")
                try:
                    # description = re.search('\)*,*.*', line).group(1)
                    # description = re.search('\),+.*', line).group(1)
                    # description = re.search('\]\s.*\)(.*)', line).group(1)
                    description = re.search('\]\](\)*,*.*)', line).group(1)
                except Exception as e:
                    print("#################################")
                    print("#### description parse error ####")
                    print(e)
                    print(index, line)
                    filepath.write(line + '\n')
                    print("#################################")
                # print(this_sec)
                # print(sub_sec)
                section_dict[sec].append({
                    'poet': poet,
                    'date': date,
                    'description': description
                })
        else:
            #print(sec)
            if re.match('===\w', line[0:4]):
                sub_sec = line.strip().replace("=", "")
                #print(sub_sec)
            if line.strip().replace("=", "") in section_dict[sec]:
                #print(sub_sec)
                this_sec[sub_sec] = []
            if sub_sec != '':
                if re.match('\*\[\[', line[0:3]):
                    try:
                        poet = re.search(r'\[\[(.*?)\]\]', line).group(1)
                        #print(poet)
                    except:
                        print("##########################")
                        print("#### poet parse error ####")
                        print(index, line)
                        filepath.write(line + '\n')
                        print(line[-2:])
                        print("##########################")
                    date_m = re.search('\((.*)\)', line)
                    try:
                        date = date_m.group(0)
                    except Exception as e:
                        print("##########################")
                        print("#### date parse error ####")
                        print(e)
                        print(index, line)
                        filepath.write(line + '\n')
                        date = ""
                        print("##########################")
                    try:
                        # description = re.search('\)*,*.*', line).group(1)
                        # description = re.search('\),+.*', line).group(0)
                        # description = re.search('\]\s.*\)(.*)', line).group(0)
                        description = re.search('\]\](\)*,*.*)', line).group(1)

                    except Exception as e:
                        print("#################################")
                        print("#### description parse error ####")
                        print(e)
                        print(index, line)
                        filepath.write(line + '\n')
                        description = ""
                        print("#################################")

                    this_sec[sub_sec].append({
                        'poet': poet,
                        'date': date,
                        'description': description
                    })
